# Projeto Análise de Qualidade de Alimentos em Plantações Agrícola com IA

O objetivo é fazer o fine-tuningem um modelo Vision Transformer Pré-treinado  e  ajustá-lo  ao  nosso  próprio  caso  de  uso, a  fim  de  classificar  e  prever  a  qualidade  de alimentos  em  plantações  agrícolas. 

Com base em uma imagem de folha, o  objetivo desta tarefa é prever o tipo de doença (Mancha Angular e Ferrugem do Feijão), se houver.Os termos em inglês são Angular Leaf Spot eBean Rust.

Fonte dos dados: https://huggingface.co/datasets/beans

## 1. Instalando e carregando os pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Ocultas avisos do Tensorflow
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [3]:
# Instala Torch
!pip install -q torch==2.0.1

In [4]:
# Instala Transformers
!pip install -q transformers==4.30.2

In [5]:
# Instala datasets
!pip install -q datasets==2.11.0

In [6]:
# Instala accelerate
!pip install accelerate

In [7]:
# Imports

from PIL import Image
import requests
import accelerate
import torch
import datasets
import transformers
import numpy as np
from datasets import load_dataset, load_metric
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto Análise Imagens Agrícolas com IA" --iversions

Author: Projeto Análise Imagens Agrícolas com IA

torch       : 2.0.1
datasets    : 2.11.0
requests    : 2.28.1
numpy       : 1.21.5
PIL         : 9.2.0
transformers: 4.30.2
accelerate  : 0.21.0



## 2. Aplicando o Fine-Tunning ao Modelo
Pré-processamento para processar os dados do modelo

### 2.1 Carga do dataset no disco

In [9]:
# Carrega os dados
dados = load_dataset('beans')

Found cached dataset beans (C:/Users/morai/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# Visualiza os dados
print(dados)

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 1034
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 133
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 128
    })
})


In [11]:
# Extrai os labels
labels = dados['train'].features['labels']

# Visualiza os dados
print(labels)

ClassLabel(names=['angular_leaf_spot', 'bean_rust', 'healthy'], id=None)


### 2.2 Aplicando o ViT Feature Extractor para Processar as Imagens

O ViT Feature Extractor serve para transformar imagens de entrada em representações vetoriais de alto nível que podem ser usadas para uma variedade de tarefas além da classificação de imagens.

In [12]:
# Repositório do ViT pré-treinado
repo_id = 'google/vit-base-patch16-224-in21k'

In [13]:
# Importa o ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained(repo_id)

In [14]:
# Visualiza o FeatureExtractor
print(feature_extractor)

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



In [15]:
# Função para o mapeamento de lotes de imagens e alpicação do ViTFeatureExtractor
def transform(example_batch):
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors = 'pt')
    inputs['labels'] = example_batch['labels']
    return inputs

In [16]:
# Prepara dos dados
prepared_data = dados.with_transform(transform)

In [17]:
# Visualiza um exemplo para conferir se a função está correta
prepared_data['train'][0:2]

{'pixel_values': tensor([[[[-0.5686, -0.5686, -0.5608,  ..., -0.0275,  0.1922, -0.2549],
          [-0.6078, -0.6000, -0.5843,  ..., -0.0353, -0.0196, -0.2706],
          [-0.6314, -0.6314, -0.6157,  ..., -0.2392, -0.3647, -0.2314],
          ...,
          [-0.5373, -0.5529, -0.5765,  ..., -0.0745, -0.0431, -0.0980],
          [-0.5608, -0.5765, -0.5843,  ...,  0.3176,  0.1608,  0.1294],
          [-0.5843, -0.5922, -0.6078,  ...,  0.2784,  0.1451,  0.2000]],

         [[-0.7098, -0.7098, -0.7490,  ..., -0.3725, -0.1608, -0.6000],
          [-0.7333, -0.7333, -0.7569,  ..., -0.3569, -0.3176, -0.5608],
          [-0.7490, -0.7490, -0.7647,  ..., -0.5373, -0.6627, -0.5373],
          ...,
          [-0.7725, -0.7882, -0.8196,  ..., -0.2314, -0.0353,  0.0824],
          [-0.7961, -0.8118, -0.8118,  ...,  0.1843,  0.3176,  0.3725],
          [-0.8196, -0.8196, -0.8275,  ...,  0.0745,  0.2863,  0.3961]],

         [[-0.9922, -0.9922, -1.0000,  ..., -0.5451, -0.3647, -0.7333],
          [-0

In [18]:
# Função para combinar as amostras
# A função combina múltiplas amostras em um único lote para o processamento do Pytorch

def collate_fn(batch):
    
    return{'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
          'labels': torch.tensor([x['labels'] for x in batch])}

## 3. Construção do Módulo de Treino do ViT

In [19]:
# Métrica do modelo
metric = load_metric('accuracy')

In [20]:
# Cálculo da métrica
def compute_metrics(prediction):
    return metric.compute(predictions = np.argmax(prediction.predictions, axis = 1),
                         references = prediction.label_ids)

In [21]:
# Copia os labels
labels = dados['train'].features['labels'].names

In [22]:
# Visualiza os labels
print(labels)

['angular_leaf_spot', 'bean_rust', 'healthy']


In [23]:
# Importa o modelo ViTForImageClassification indicandos os novos labels que serão usados
modelo = ViTForImageClassification.from_pretrained(repo_id,
                                                  num_labels = len(labels),
                                                  id2label = {str(i):c for i, c in enumerate(labels)},
                                                  label2id = {c:str(i) for i, c in enumerate(labels)})

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Esta mensagem acima está dizendo que você está inicializando um modelo ViTForImageClassification (um modelo de classificação de imagem Vision Transformer) a partir de um modelo pré-treinado (google/vit-base-patch16-224-in21k). No entanto, nem todos os pesos do modelo pré-treinado estão sendo usados na inicialização e alguns pesos do modelo ViTForImageClassification estão sendo inicializados do zero.

Há dois pontos principais aqui:

Alguns pesos do checkpoint do modelo ('pooler.dense.bias', 'pooler.dense.weight') não foram utilizados na inicialização do ViTForImageClassification. Isso pode ser esperado se você estiver inicializando o ViTForImageClassification a partir do checkpoint de um modelo treinado em outra tarefa ou com outra arquitetura. Se o modelo pré-treinado fosse exatamente idêntico à arquitetura que você está inicializando, você esperaria que todos os pesos fossem usados, e a mensagem informaria que algo está errado se isso não acontecesse.

Alguns pesos do ViTForImageClassification ('classifier.bias', 'classifier.weight') não foram inicializados a partir do checkpoint do modelo e foram recém-inicializados. Isso significa que esses componentes específicos do modelo não receberam pesos do modelo pré-treinado e, em vez disso, foram inicializados, provavelmente com alguma forma de inicialização aleatória.

A mensagem termina sugerindo que você provavelmente deve treinar (ou seja, fazer um "fine-tuning") este modelo em uma tarefa antes de usá-lo para predições e inferências. Isso ocorre porque, embora o modelo tenha sido parcialmente inicializado com pesos de um modelo pré-treinado, ele ainda tem alguns pesos que foram inicializados aleatoriamente e, portanto, precisam ser ajustados para a tarefa específica que você deseja resolver.

In [24]:
# Argumentos de treino
training_args = TrainingArguments(output_dir = "resultados",
                                  evaluation_strategy = 'steps',
                                  num_train_epochs = 4,
                                  learning_rate = 2e-4,
                                  remove_unused_columns = False,
                                  load_best_model_at_end = True)

In [25]:
# Trainer
trainer = Trainer(model = modelo,
                  args = training_args,
                  data_collator = collate_fn,
                  compute_metrics = compute_metrics,
                  train_dataset = prepared_data['train'],
                  eval_dataset = prepared_data['validation'],
                  tokenizer = feature_extractor)

## 4. Treino do modelo

In [26]:
%%time
train_results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.148500,0.052046,0.977444


Wall time: 57min 29s


In [27]:
# Salva o modelo em disco
trainer.save_model('modelos')

In [30]:
# Log das métricas
trainer.log_metrics('train', train_results.metrics)

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 298497957GF
  train_loss               =       0.143
  train_runtime            =  0:57:29.79
  train_samples_per_second =       1.199
  train_steps_per_second   =       0.151


In [31]:
# Salva as métricas
trainer.save_metrics('train', train_results.metrics)

## 5. Avaliação do modelo

In [32]:
# Avaliação do modelo 
metrics = trainer.evaluate(prepared_data['validation'])
trainer.log_metrics('eval', metrics)
trainer.save_metrics('eval', metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9774
  eval_loss               =      0.052
  eval_runtime            = 0:00:37.65
  eval_samples_per_second =      3.532
  eval_steps_per_second   =      0.451


## 6. Deploy do modelo

In [33]:
# URL de uma imagem (experimente outras imagens)
url = 'https://www.greenlife.co.ke/wp-content/uploads/2022/04/disease_bean_angular_leaf_spot.jpg'

In [34]:
# Carrega a imagem
image = Image.open(requests.get(url, stream = True).raw)

In [35]:
# Aplica o extrator
inputs = feature_extractor(images = image, return_tensors = "pt")

In [36]:
# Coloca a imagem no mesmo device do modelo
inputs = {name: tensor.to(trainer.args.device) for name, tensor in inputs.items()}

In [37]:
# Visualiza os inputs
inputs

{'pixel_values': tensor([[[[ 0.6157,  0.6078,  0.6314,  ..., -0.0118, -0.0275, -0.0353],
           [ 0.6000,  0.6000,  0.6078,  ..., -0.0118, -0.0431, -0.0667],
           [ 0.6000,  0.6157,  0.5922,  ..., -0.0039, -0.0196, -0.0824],
           ...,
           [ 0.4353,  0.4039,  0.4118,  ...,  0.0275,  0.0039,  0.0118],
           [ 0.4275,  0.4196,  0.4353,  ...,  0.0431,  0.0039,  0.0039],
           [ 0.4039,  0.4039,  0.4275,  ...,  0.0431,  0.0118, -0.0118]],
 
          [[ 0.4118,  0.4275,  0.4510,  ..., -0.1294, -0.1451, -0.1529],
           [ 0.3961,  0.4275,  0.4275,  ..., -0.1137, -0.1529, -0.1765],
           [ 0.4118,  0.4353,  0.4118,  ..., -0.1137, -0.1216, -0.2000],
           ...,
           [ 0.2000,  0.1922,  0.1922,  ..., -0.0980, -0.1137, -0.1059],
           [ 0.1843,  0.2078,  0.2157,  ..., -0.0824, -0.1294, -0.1216],
           [ 0.1608,  0.2235,  0.2235,  ..., -0.0824, -0.1373, -0.1294]],
 
          [[ 0.0353,  0.0353,  0.0902,  ..., -0.4824, -0.4902, -0.4824

In [38]:
# Coloca o modelo em modo de avaliação
trainer.model.eval() 

# Desliga os gradientes para a inferência
with torch.no_grad():  
    outputs = trainer.model(**inputs)

In [39]:
# Extrai os logits
logits = outputs.logits  

In [40]:
# Extrai o logit de maior valor para a imagem
class_index = logits.argmax()

In [41]:
# Extrai o valor que desejamos do tensor
valor = class_index.item()
valor

0

In [42]:
# Define o mapeamento original (está na documentação do dataset bean)
mapping = {
  "angular_leaf_spot": 0,
  "bean_rust": 1,
  "healthy": 2,
}

In [43]:
# Cria um mapeamento reverso
reverse_mapping = {v: k for k, v in mapping.items()}
reverse_mapping

{0: 'angular_leaf_spot', 1: 'bean_rust', 2: 'healthy'}

In [44]:
# Usa o mapeamento reverso para obter o nome da classe
class_name = reverse_mapping.get(valor)

In [45]:
print("A imagem foi classificada como:", class_name)

A imagem foi classificada como: angular_leaf_spot
